In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedGroupKFold, StratifiedKFold
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.utils.class_weight import compute_class_weight

import textstat
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

import wandb

## Выбор кейса 

В данной работе я ставлю перед собой следущую бизнес-цель: решить задачу **бинарной классификации вакансий на настоящие и мошеннические** для антифрод-системы сервиса с объявлениями о работе. В данной задаче классы сильно несбалансированны, так как мошеннических вакансий обычно существенно меньше, чем настоящих.

## Сбор данных

### Датасет для исходной задачи

Для своей задачи я нашел два подходящих датасет [Real / Fake Job Posting Prediction](https://www.kaggle.com/datasets/shivamb/real-or-fake-fake-jobposting-prediction) с 18 тыс. вакансий, из которых около 800 являются фейковыми

Датасет имеет лицензию [CC0: Public Domain](https://creativecommons.org/publicdomain/zero/1.0/), поэтому его можно использовать для обучающих, исследовательских и коммерческих целей, не спрашивая разрешения у авторов.

Загрузим датасет и прочитаем его

In [2]:
# !mkdir data/real_fake_postings/ && cd data/real_fake_postings/ && kaggle datasets download shivamb/real-or-fake-fake-jobposting-prediction && unzip real-or-fake-fake-jobposting-prediction.zip 

In [3]:
real_fake_df = pd.read_csv("data/real_fake_postings/fake_job_postings.csv")

In [4]:
real_fake_df.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


### Weak Supervision

#### Датасет из смежной задачи
Из смежной задачи, которая заключается в предсказании зарплаты по данным вакансии, я нашёл датасет [LinkedIn Job Postings (2023 - 2024)](https://www.kaggle.com/datasets/arshkon/linkedin-job-postings/data) с более 100 тыс. вакансиями из LinkedIn. Датасет имеет лицензию [CC BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/), поэтому его можно использовать для исследовательских и коммерческих целей, но выпуская наш продукт под той же лицензией с указанием ссылки на исходный датасет.

В этом датасете нет информации о том, является ли заданное объявляение фейком или нет, но я воспользуюсь Weak Supervision, чтобы доразметить его. В частности, я хочу излвечь оттуда явно фейковые вакансии, что поможет увеличить датасет и уменьшить дисбаланс классов.


#### Подход для доразметки
Я буду помечать вакансию как фейковую, если в её описании встерчаются определенные фразы. Например, "No experience required", "Unlimited earning potential", "Financial freedom in weeks". После чего из таких вакансий выберу top-N с самыми короткими описаниями профилей компаний (фейковые компании обычно имеют не очень развернутые описания).

Загрузим датасет и найтем те объявляения, которые, скорее всего, являются фейковыми

In [5]:
unlabeled_postings = pd.read_csv("data/linkedin_postings/postings.csv")
companies = pd.read_csv("data/linkedin_postings/companies/companies.csv")[["company_id", "description"]]
unlabeled_postings = pd.merge(left=unlabeled_postings, right=companies, how="left", on="company_id", suffixes=["", "_company"])
unlabeled_postings.head()

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,normalized_salary,zip_code,fips,description_company
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,20.0,HOURLY,"Princeton, NJ",2774458.0,20.0,NaN,...,1.713398e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,38480.0,8540.0,34021.0,With years of experience helping local buyers ...
1,1829192,NaN,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",50.0,HOURLY,"Fort Collins, CO",NaN,1.0,NaN,...,1.712858e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,83200.0,80521.0,8069.0,NaN
2,10998357,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,65000.0,YEARLY,"Cincinnati, OH",64896719.0,8.0,NaN,...,1.713278e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,55000.0,45202.0,39061.0,"In April of 1983, The National Exemplar began ..."
3,23221523,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,175000.0,YEARLY,"New Hyde Park, NY",766262.0,16.0,NaN,...,1.712896e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,157500.0,11040.0,36059.0,"Abrams Fensterman, LLP is a full-service law f..."
4,35982263,NaN,Service Technician,Looking for HVAC service tech with experience ...,80000.0,YEARLY,"Burlington, IA",NaN,3.0,NaN,...,1.713452e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,70000.0,52601.0,19057.0,NaN


In [6]:
import re

def is_fraudulent_job_description(text):
    red_flag_patterns = [
        r'\b(earn\s*\$\d+[\d,.]*\s*(per|/)\s*(month|week|hour|yr|year)|make \$[\d,.]+\s+(fast|quick))\b',
        r'\b(pay\s*(a|the)\s+fee|upfront\s+cost|security\s+deposit|required\s+investment)\b',
        r'\b(no\s+experience\s+required|no\s+qualifications\s+needed)\b',
        r'\b(guaranteed\s+income|financial\s+freedom\s+in\s+\d+\s+weeks)\b',
        r'\b(multi[\s-]*level\s+marketing|mlm|pyramid\s+scheme)\b',
        r'\b(send\s+(your\s+)?(personal\s+)?(information|details|bank\s+account|ssn|social\s+security))\b',
        r'\b(work\s+from\s+home\s+(with\s+)?no\s+(experience|interview))\b',
        r'\b(cryptocurrency\s+investment|bitcoin\s+mining|process\s+payments)\b',
        r'\b(recruit\s+\d+\s+people|build\s+your\s+team|referral\s+commissions)\b',
        r'\b(urgently\s+hiring|immediate\s+start|positions?\s+available\s+now)\b',
        r'\b(kindly\s+send|dear\s+candidate,|this\s+is\s+not\s+a\s+scam)\b',
        r'\b(free\s+training\s+materials|company\s+will\s+send\s+you\s+a\s+check)\b',
        r'\b(government\s+approved|100%\s+legitimate|risk-free\s+opportunity)\b',
        r'\b(wire\s+transfers|international\s+transactions|money\s+transfer)\b',
    ]

    combined_pattern = re.compile(
        '(' + '|'.join(red_flag_patterns) + ')',
        flags=re.IGNORECASE
    )

    return bool(combined_pattern.search(text))

In [7]:
is_fraud_job_desc = unlabeled_postings.description.fillna("").apply(lambda desc: is_fraudulent_job_description(desc))

suspicious_jobs = unlabeled_postings[is_fraud_job_desc]
suspicious_jobs.loc[:, "description_company_len"] = unlabeled_postings[is_fraud_job_desc].description_company.fillna("").apply(lambda desc: len(desc))

suspicious_jobs = suspicious_jobs[suspicious_jobs.company_id.isna() | ~suspicious_jobs.company_id.duplicated()]
topN = 300
suspicious_jobs_idx = suspicious_jobs.description_company_len.sort_values(ascending=True)[:topN].index
suspicious_jobs_final = suspicious_jobs.loc[suspicious_jobs_idx]

/tmp/ipykernel_1680271/2360352750.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  suspicious_jobs.loc[:, "description_company_len"] = unlabeled_postings[is_fraud_job_desc].description_company.fillna("").apply(lambda desc: len(desc))


In [8]:
suspicious_jobs_final.head()

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,posting_domain,sponsored,work_type,currency,compensation_type,normalized_salary,zip_code,fips,description_company,description_company_len
23561,3889766866,NaN,Vice President Finance,Reports To: CEOFLSA: ExemptLocation: Remote Wh...,200000.0,YEARLY,"Shirley, NY",NaN,33.0,NaN,...,NaN,0,FULL_TIME,USD,BASE_SALARY,190000.0,11967.0,36103.0,NaN,0
23618,3889770622,NaN,Treasury Analyst,Overview of the Position:\nFull time position ...,NaN,NaN,New York City Metropolitan Area,NaN,11.0,NaN,...,NaN,0,FULL_TIME,NaN,NaN,NaN,NaN,NaN,NaN,0
23302,3889751726,NaN,Gift Shop Manager,Gift Shop Manager The White House Historical A...,65000.0,YEARLY,"Washington, DC",NaN,6.0,NaN,...,NaN,0,FULL_TIME,USD,BASE_SALARY,60000.0,20001.0,11001.0,NaN,0
36952,3895599731,A Hiring Company,Kitchen Leader,Go Chicken Go Hiring Now!\n\n \n\nAt Go Chicke...,NaN,NaN,"Missouri City, TX",101478385.0,2.0,NaN,...,www.click2apply.net,0,FULL_TIME,NaN,NaN,NaN,77459.0,48157.0,NaN,0
118020,3906085362,NaN,Unemployed,Remote Positions Available!!!\nFinancial offic...,NaN,NaN,United States,NaN,11.0,NaN,...,NaN,0,OTHER,NaN,NaN,NaN,NaN,NaN,NaN,0


### Метрика качества и тестовый датасет
#### Метрика качества
Наша задача -- бинарная классификация с дисбалансом классов, поэтому выберем интепретируемые метрики, которые устойчивы к дисбалансу классов:
1. F1-score
2. Precision
3. Recall

Отметим, что мы не выбрали AUC-ROC по причине того, что в задачах, где не так важен больший класс, он может давать не совсем адекватную картину при сравнении алгоритмов.

В нашей задаче важен как precision (не хотим банить настоящие объявления), так и recall (не хотим пропускать мошеннические объявления), поэтому в качестве основной метрики возьмем **f1-score**, так как она балансирует между предыдущими двумя. Precision и Recall будут второстепенными метриками.



#### Подготовка данных перед разбиением на трейн и тест

Хочется, чтобы модель не требовала слишком много входной информации, потому что в таком случае ее будет удобнее использовать. Поэтому я оставлю только доступные пользователю и самые, по моему мнению, важные для детектирования фейка признаки: название и описание вакансии, описание компании.

In [9]:
y = real_fake_df.fraudulent
features = ["title", "description", "company_profile"]
X = real_fake_df[features]

У нас три текстовые колонки, предобработаем их.

In [10]:
def preprocess_text(text):
    """Comprehensive text preprocessing"""
    if pd.isna(text):
        return ""
    text = str(text).lower().strip()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\d+', '', text)
    return text


processed_features = []
for feat in features:
    X[feat] = X[feat].fillna("")
    new_feat = feat + "_processed"
    processed_features.append(new_feat)
    X[new_feat] = X[feat].apply(preprocess_text)

/tmp/ipykernel_1680271/1874289439.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[feat] = X[feat].fillna("")
/tmp/ipykernel_1680271/1874289439.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[new_feat] = X[feat].apply(preprocess_text)
/tmp/ipykernel_1680271/1874289439.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

Для начала проверим нет ли в датасете дубликатов

In [11]:
X.duplicated(subset=processed_features, keep='first').sum()

2308

Они есть, поэтому уберем их

In [12]:
not_dupl_mask = ~X.duplicated(subset=processed_features, keep="first")
X = X[not_dupl_mask].reset_index(drop=True)
y = y[not_dupl_mask].reset_index(drop=True)

Также посмотрим на очень похожие объекты и по возможности от них избавимся

In [13]:
X_new = X.copy()
y_new = y.copy()


X_new['combined'] = (
    X_new['title_processed'] + ' ' +
    X_new['description_processed'] + ' ' +
    X_new['company_profile_processed']
)


tfidf = TfidfVectorizer(stop_words='english', min_df=0.01, max_df=0.85)
tfidf_matrix = tfidf.fit_transform(X_new['combined'])
cosine_sim = cosine_similarity(tfidf_matrix)

threshold = 0.95
duplicates = set()

for i in range(cosine_sim.shape[0]):
    if i not in duplicates:
        similar = np.where(cosine_sim[i] > threshold)[0]
        similar = [s for s in similar if s != i and s not in duplicates]
        duplicates.update(similar)

X_new_clean = X_new.drop(index=list(duplicates), columns="combined").reset_index(drop=True)
y_new_clean = y_new.drop(index=list(duplicates), columns="combined").reset_index(drop=True)

#### Разбиение датасета на обучающий и тестовый
Нам не подходит случайное разбиение на трейн и тест, так как, скорее всего, вакансии, принадлежащие отной компании, либо все фейковые, либо все настоящие. Поэтому нельзя допустить, чтобы вакансии одной компании были одновременно и в трейне и в тесте (иначе будет утечка).

Именно поэтому мы будем разбивать так, что строки с одинаковым описанием компании обязательно пойдут в ровно одну из выборок. Также по возможности будем поддерживать долю положительного класса приблизтельно одинаковой в выборках. Для этого воспользуемся `StratifiedGroupKFold`

In [14]:
company_profile_to_group = {desc: i for i, desc in enumerate(X_new_clean.company_profile.unique().tolist(), 0)}
groups = X_new_clean.company_profile.apply(lambda prof: company_profile_to_group[prof])
n_splits = int(1 / 0.3)

splitter = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state=42)
train_idx, test_idx = next(splitter.split(X_new_clean, y_new_clean, groups))

# Split the data
X_train, X_test = X_new_clean.iloc[train_idx], X_new_clean.iloc[test_idx]
y_train, y_test = y_new_clean.iloc[train_idx], y_new_clean.iloc[test_idx]
groups_train = groups.iloc[train_idx]


print(f"Train target mean: {y_train.mean():.4f}")
print(f"Test target mean: {y_test.mean():.4f}")
print(f"Original target mean: {y_new_clean.mean():.4f}")

print(f"Test share: {X_test.shape[0] / X_new_clean.shape[0]}")

Train target mean: 0.0498
Test target mean: 0.0386
Original target mean: 0.0475
Test share: 0.20795932957343713


Видно, что доли положительного класса приблизительно одинаковы

### Подготовка к обучению

#### Добавление Weak Supervision датасета в обучающую выборку
Ранее с помощью Weak Supervision из другого датасета мы получили объявления о работе, которые мы будем считать фейковыми. Добавим их в нашу обучающую выборку, преобразуя по возможности столбцы одного датасета в соотвествующие столбцы другого.

In [15]:
new_X_train_part = suspicious_jobs_final.copy()
old_feature_to_new = {
    "title": "title",
    "description": "description",
    "company_profile": "description_company"
}

for feat in features:
    new_X_train_part[feat] = new_X_train_part[old_feature_to_new[feat]].fillna("")
    new_feat = feat + "_processed"
    new_X_train_part[new_feat] = new_X_train_part[feat].apply(preprocess_text)

new_X_train_part = new_X_train_part[features + processed_features]
new_y_train_part = pd.Series(np.ones((new_X_train_part.shape[0])))

X_train = pd.concat((X_train, new_X_train_part)).reset_index(drop=True)
groups_train = pd.concat([groups_train, pd.Series(np.arange(groups_train.max() + 1, len(new_X_train_part) + groups_train.max() + 1))]).reset_index(drop=True)
y_train = pd.Series(pd.concat((y_train, new_y_train_part)).reset_index(drop=True), name='fraudulent')

Убедимся, что в трейне дубликатов не добавилось

In [16]:
X_train.duplicated().sum()

0

Сохраним данные в файлики

In [17]:
X_train.to_csv("data/my/before_feature_engineering/train.csv", sep=",", header=True)
X_test.to_csv("data/my/before_feature_engineering/test.csv", sep=",", header=True)

y_train.to_csv("data/my/before_feature_engineering/train_labels.csv", sep=",", header=True)
y_test.to_csv("data/my/before_feature_engineering/test_labels.csv", sep=",", header=True)

groups_train.to_csv("data/my/before_feature_engineering/train_groups.csv", sep=",", header=True)


#### Преобразование данных

Все необходимые преобразования исходных данных мы провели до разбиения на трейн и тест. Теперь займемся придумыванием новых признаков. Воспользуемся следующими методами:
- с помощью tf-idf построим векторное представление текстовых признаков, далее уменьшим их размерность с помощью PCA 
- посчитаем некоторые текстовые статистики, которые могут быть полезны для детекстирования фрода. Например, длина текста может быть важная -- пустые описания часто характерны для фейковых вакансий

In [18]:
class TextStatsTransformer(BaseEstimator, TransformerMixin):
    def _download_nltk_resources(self):
        try:
            nltk.data.find('sentiment/vader_lexicon.zip')
        except LookupError:
            print("Downloading NLTK VADER lexicon...")
            nltk.download('vader_lexicon', quiet=False)
    
    
    def __init__(self):
        self._download_nltk_resources()
        self.sia = SentimentIntensityAnalyzer()
        
    
    def _process_column(self, X):
        stats = pd.DataFrame(index=X.index)
        text = X.fillna('').apply(preprocess_text)
        
        stats['char_count'] = text.apply(len)
        stats['word_count'] = text.apply(lambda x: len(x.split()))
        stats['unique_words'] = text.apply(lambda x: len(set(x.split())))
        stats['readability'] = text.apply(textstat.flesch_reading_ease)
        stats['sentiment'] = text.apply(lambda x: self.sia.polarity_scores(x)['compound'])
        stats['has_url'] = text.str.contains(r'http[s]?://').astype(int)
        
        stats['exclamation_count'] = text.str.count(r'!')
        stats['all_caps_ratio'] = text.apply(
            lambda x: sum(1 for w in x.split() if w.isupper()) / len(x.split()) 
            if len(x.split()) > 0 else 0
        )
        
        return stats

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return self._process_column(X)


def create_tfidf_pipeline(max_features=1000, n_components=50):
    return Pipeline([
        (
            'tfidf',
            TfidfVectorizer(
                preprocessor=preprocess_text,
                max_features=max_features,
                stop_words='english',
                ngram_range=(1, 2),
                analyzer='word'
            )
        ),
        (
            'to_dense', 
            FunctionTransformer(
                lambda x: x.toarray(), 
                accept_sparse=True
            ),
        ),
        (
            'pca', 
            PCA(
                n_components=n_components,
                whiten=True,
                random_state=42
            )
        )
    ])


tfidf_features = ColumnTransformer([
    ('title_tfidf', create_tfidf_pipeline(500, 5), 'title'),
    ('desc_tfidf', create_tfidf_pipeline(2000, 75), 'description'),
    ('profile_tfidf', create_tfidf_pipeline(2000, 75), 'company_profile')
])

stats_features = ColumnTransformer([
    (f'{col}_stats', TextStatsTransformer(), col)
    for col in features
])

final_pipeline = Pipeline([
    (
        'features', 
        FeatureUnion([
            ('tfidf', tfidf_features),
            ('stats', stats_features)
        ])
    ),
    (
        'scaler', 
        StandardScaler()
    )
])


In [19]:
X_train_tr = pd.DataFrame(final_pipeline.fit_transform(X_train))
X_test_tr = pd.DataFrame(final_pipeline.transform(X_test))

In [20]:
X_train_tr.to_csv("data/my/after_feature_engineering/train.csv", sep=",", header=True)
X_test_tr.to_csv("data/my/after_feature_engineering/test.csv", sep=",", header=True)

#### Проверка на утечку данных

Кажется, утечки данных не происходит, так как:
1. Мы предсказываем фейковость объявления по его названию, описаниям компании и работы. В этих признаках нет явной утечки таргета с точки зрения здравого смысла
2. Дедупликация данных была произведена до разбиения на трейн и тест (похожие объекты могут вызвать утечку)
3. Feature engineering делается только по обучающей выборке, а к тестовой только применяется (fit_transform/transform)
4. Мы не использовали информацию о таргете в предобработке данных/фича инжиниринге
5. С потенциальной утечкой в дубликаций описаний компании мы поборолись при разбиении на трейн и тест

### Бейзлайн

В качестве бейзлайна я возьму логистическую регрессию, так как:
- Она интерпретируема, так как коэффициенты отражают важность признаков (т.к. мы их пошкалировали)
- Она быстро учится, в сравнение с многими другими алгоритмами
- Хорошо работает с большим количеством признаков (актуально, так как у нас много tf-idf признаков)
- На выход выдает вероятность, что позволяет ее удобно калибровать (особенно важно для нашей задачи с явным precision/recall трейдоффом)

Другие варианты бейзлайна и причины, почему их я не выбрал:
1. Наивный баес
    - Плохо работает с смешанными типами фичей (в нашем случае tf-idf и статистики)
    - Плохо работает с зависимыми признаками
2. Решающее дерево
    - Переобучается, в особенности под редкие n-граммы
3. Случайный лес
    - Медленее учится
    - Менее интепретируем
    - Лучше его попробовать после линейной регрессии
4. Нейронные сети / градиентный бустинг
    - оверкилл для бейзлайна

### Обучение первой модели

Обучим логистическую регрессию, а аткже подберем коэффициент регуляризации и вес класса по кросс-валидации. Будем логировать метрикики в wandb

In [22]:
model = LogisticRegression(
    class_weight='balanced',
    random_state=42,
    max_iter=1000
)

with wandb.init(project="job-fake-prediction", 
          config={
              "model_type": "logistic_regression",
              "validation": "stratified_kfold",
              "k_folds": 5
          }):
    classes = np.unique(y_train)
    balanced_weights = compute_class_weight('balanced', classes=classes, y=y_train)
    class_ratio = balanced_weights[1]/balanced_weights[0]

    class_weight_grid = [
        None,
        'balanced',
        {0: 1, 1: 10},
        {0: 1, 1: 30},
        {0: 1, 1: 50},
        {0: 1, 1: int(class_ratio*1.5)},
        {0: 1, 1: int(class_ratio*3)}
    ]

    param_grid = {
        'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
        'class_weight': class_weight_grid
    }
    
    skf = StratifiedKFold(n_splits=5)
    best_score = 0
    best_params = {}

    for c in param_grid['C']:
        for class_weight in param_grid['class_weight']:
            fold_metrics = {'f1': [], 'precision': [], 'recall': []}
            
            wandb.log({
                'C': c,
                'class_weight': str(class_weight),
                'status': 'started'
            }, commit=False)
            
            for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_tr, y_train)):
                X_train_f, X_val_f = X_train_tr.iloc[train_idx], X_train_tr.iloc[val_idx]
                y_train_f, y_val_f = y_train.iloc[train_idx], y_train.iloc[val_idx]
                
                model_f = model.set_params(
                    C=c,
                    class_weight=class_weight
                )
                
                model_f.fit(X_train_f, y_train_f)
                y_pred = model_f.predict(X_val_f)
                
                fold_metrics['f1'].append(f1_score(y_val_f, y_pred))
                fold_metrics['precision'].append(precision_score(y_val_f, y_pred))
                fold_metrics['recall'].append(recall_score(y_val_f, y_pred))
                
                wandb.log({
                    'C': c,
                    'class_weight': str(class_weight),
                    'fold': fold + 1,
                    'fold_f1': fold_metrics['f1'][-1],
                    'fold_precision': fold_metrics['precision'][-1],
                    'fold_recall': fold_metrics['recall'][-1]
                })
            
            mean_metrics = {
                'mean_f1': np.mean(fold_metrics['f1']),
                'mean_precision': np.mean(fold_metrics['precision']),
                'mean_recall': np.mean(fold_metrics['recall'])
            }
            
            wandb.log({
                **mean_metrics,
                'C': c,
                'class_weight': str(class_weight),
                'status': 'completed'
            })
            
            if mean_metrics['mean_f1'] > best_score:
                best_score = mean_metrics['mean_f1']
                best_params = {'C': c, 'class_weight': class_weight}
            wandb.log({"best_mean_f1": best_score})
                
    wandb.log({
        'best_C': best_params['C'],
        'best_class_weight': str(best_params['class_weight']),
        'best_mean_f1': best_score
    })


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: dangerio to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/home/aksenovan/my_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


C,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂████
best_C,▁
best_mean_f1,▁▇██████████████████████████████████████
fold,▃█▃▆█▁▁▅▆██▁▆█▃▅█▁▁▁▆▃▃▁▆▅▆▁▆█▁▅█▆█▆▆▃██
fold_f1,▆▅▆▃▂▁█▅▆▄▄▅▄▅▄▃█▄▄▆▃▅█▄▄▄▅▄▄▃▅▄▃▄▅▅▅▅▆▆
fold_precision,▂▂▃▄▁▂▁▂▃▄▂▃█▂▂▂▃▃█▆▃▃▂▂▂▄▁▁▂▂▂▂▇▁▂▆▂▃▂▂
fold_recall,█▇█▃▂█▇▇█▇▇▇▁▆▇▁▂▇▆▆▇▂▆▅█▇▇█▂▂▇▆▆▇▆▅▇▇▇▇
mean_f1,▁▇█▆▅▆▆██▇▇▆███▆▇▇██▆▇▇██▇▆▇▇██▇▆▇▇██▇▆▇
mean_precision,▅▂▃▁▁▁█▃▃▂▂▂█▃▃▂▂▂▇▃▂▂▂▂▇▃▂▂▂▂▃▃▂▂▂▃▃▂▂▂
mean_recall,▁▇▆██▂▇▇▇█▇▃▇▆▇▇▇▃▇▆▇▇▇▃▇▇▇▇▇▃▆▇▇▇▇▇▆▇▇▇
C,1000


Скрины метрик (step здесь имеет смысл очередного сплита кросс-валидации):

![](images/baseline/f1.png)
![](images/baseline/precision.png)
![](images/baseline/recall.png)

Теперь обучим с наилучшими параметрами логистическую регрессию на всем трейне и посчитаем метрики на трейне и тесте.

In [23]:
best_model = model = LogisticRegression(
    random_state=42,
    max_iter=1000,
    **best_params
)

best_model.fit(X_train_tr, y_train)

y_train_pred = best_model.predict(X_train_tr)
y_test_pred = best_model.predict(X_test_tr)

print(f"Train F1-Score: {f1_score(y_train, y_train_pred):.3f}")
print(f"Test F1-Score: {f1_score(y_test, y_test_pred):.3f}", end="\n\n")

print(f"Train Precision: {precision_score(y_train, y_train_pred):.3f}")
print(f"Test Precision: {precision_score(y_test, y_test_pred):.3f}", end="\n\n")

print(f"Train Recall: {recall_score(y_train, y_train_pred):.3f}")
print(f"Test Recall: {recall_score(y_test, y_test_pred):.3f}")

Train F1-Score: 0.500
Test F1-Score: 0.237

Train Precision: 0.350
Test Precision: 0.161

Train Recall: 0.876
Test Recall: 0.446


На глаз результаты кажутся неплохими -- мы выявлили почти половину фродовых вакансий, правда ошибаясь в 74% случаев. Однако учитывая, что фрода в датасете довольно мало, в абсолютном выражении мы ошибаемся на так часто.